In [21]:
%matplotlib widget
from lcls_beamline_toolbox.models import mfx
import vonhamos_spectrometer as vh


In [ ]:
mfx_sim = mfx.MFX(E0=9000, N=256)

import os, glob
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# ---------------- CONFIG ------------------------------------------------------
offset       = 1e-6               # rad per step unit (µrad)
steps        = np.linspace(-5, 5, 15)   # 15 points
output_dir   = "pitch_scan"
gif_name     = "pitch_scan.gif"
gif_duration = 500                # ms per frame
# -----------------------------------------------------------------------------


# -------- fresh output directory ---------------------------------------------
os.makedirs(output_dir, exist_ok=True)
for f in glob.glob(os.path.join(output_dir, "frame_*.png")):
    os.remove(f)

# -------- scan MR1L4 pitch ----------------------------------------------------
pitch_vals, xcm_vals, frame_paths = [], [], []
vmin, vmax = np.inf, -np.inf
profiles   = []

for idx, step in enumerate(steps):
    sim = mfx.MFX(9500, N=256)
    #print(dir(sim))
    print(sim.mr1l4_pitch.wm())
    print(step * offset)
    sim.mr1l4_pitch.mvr(step * offset)
    sim.propagate()

    pitch = sim.mr1l4_pitch.wm()
    img   = sim.beamline.DG1_YAG.profile
    profiles.append(img)
    pitch_vals.append(pitch)

    # pixel-space centroid
    dx = sim.beamline.DG1_YAG.dx
    nx = img.shape[1]
    xcm = sim.beamline.DG1_YAG.cx / dx + nx / 2
    xcm_vals.append(xcm)

    vmin = min(vmin, img.min())
    vmax = max(vmax, img.max())

    # build frame via view_beam()
    axes = sim.beamline.DG1_YAG.view_beam()         # [profile, x, y] axes
    axes[0].set_title(f"DG1_YAG – MR1L4 Pitch: {pitch:.2e}")  # same style as TFS_Z
    fig = axes[0].figure

    frame = os.path.join(output_dir, f"frame_{idx:02d}.png")
    fig.savefig(frame, bbox_inches="tight", dpi=150)
    plt.close(fig)
    frame_paths.append(frame)

# -------- make GIF ------------------------------------------------------------
gif_path = os.path.join(output_dir, gif_name)
Image.open(frame_paths[0]).save(
    gif_path,
    save_all=True,
    append_images=[Image.open(p) for p in frame_paths[1:]],
    duration=gif_duration,
    loop=0
)
print("GIF saved →", gif_path)

# -------- quadratic fit in µrad ---------------------------------------------
pitch_vals = np.asarray(pitch_vals, dtype=float)   
xcm_vals   = np.asarray(xcm_vals,   dtype=float)
pitch_mu  = pitch_vals * 1e6          

# -------- diagnostic plot -----------------------------------------------------
plt.figure()
plt.plot(pitch_mu, xcm_vals, "o", label="data")
plt.xlabel("Pitch (µrad)")
plt.ylabel("Spot x-centroid (px)")
plt.title("DG1_YAG spot position vs MR1L4 pitch")
plt.tight_layout()
plt.savefig(os.path.join(output_dir, "pitch_fit.png"), dpi=150)
plt.close()
print("Diagnostic plot →", os.path.join(output_dir, "pitch_fit.png"))

print("Saved diagnostic plot in", output_dir)


-0.000552
-8e-06


ValueError: zero-size array to reduction operation maximum which has no identity

In [24]:
mfx_sim.propagate()
# mfx_sim.beamline.DG2_YAG.view_beam()

# mfx_ip = mfx_sim.beamline.MFX_IP
# mfx_sim.beamline.MFX_IP.view_beam()

# wx = mfx_ip.wx * 1000 / 2.355   # convert from m to mm then FWHM to sigma
# wy = mfx_ip.wy * 1000 / 2.355

In [17]:
# 2) build von Hamos spectrometer and attach source
beamLine = vh.build_beamline(energy=mfx_sim.E0, beamH=wx, beamV=wy)

# 3) run ray tracing
vh.rrun.run_process = vh.run_process
plots = vh.define_plots()
vh.xrtrun.run_ray_tracing(plots=plots, beamLine=beamLine, backend="raycing")

The last 10 runs
/Users/aminelamouchi/opt/anaconda3/envs/mini_env/lib/python3.9/inspect.py::
start: Mon, 21 Jul 2025 16:04:51; stop: 16:04:59; duration: 7.6 s
/Users/aminelamouchi/opt/anaconda3/envs/mini_env/lib/python3.9/inspect.py::
start: Mon, 21 Jul 2025 16:05:34; stop: 16:05:41; duration: 6.9 s
/Users/aminelamouchi/opt/anaconda3/envs/mini_env/lib/python3.9/inspect.py::
start: Mon, 21 Jul 2025 17:20:16; stop: 17:20:22; duration: 6.1 s
/Users/aminelamouchi/opt/anaconda3/envs/mini_env/lib/python3.9/inspect.py::
start: Mon, 21 Jul 2025 17:35:42; stop: 17:35:48; duration: 6.3 s
/Users/aminelamouchi/opt/anaconda3/envs/mini_env/lib/python3.9/inspect.py::
start: Mon, 21 Jul 2025 17:35:59; stop: 17:36:05; duration: 6.3 s
/Users/aminelamouchi/opt/anaconda3/envs/mini_env/lib/python3.9/inspect.py::
start: Mon, 21 Jul 2025 17:37:33; stop: 17:37:41; duration: 7.7 s
/Users/aminelamouchi/opt/anaconda3/envs/mini_env/lib/python3.9/inspect.py::
start: Mon, 21 Jul 2025 17:46:03; stop: 17:46:09; durat

/Users/aminelamouchi/opt/anaconda3/envs/mini_env/lib/python3.9/site-packages/xrt-1.6.1-py3.9.egg/xrt/runner.py:651: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [18]:
plot = plots[0]  # This is the XYCPlot object

cx = plot.cx
cy = plot.cy
dx = plot.dx/2
dy = plot.dy/2

print("x center: ", cx)
print("x FWHM: ", dx)
print("y center: ", cy)
print("y FWHM: ", dy)

x center:  0.02810745546078031
x FWHM:  0.3749705173225689
y center:  -0.6206777687648077
y FWHM:  9.040630524264913


In [31]:
# check which crls are in use
for crl in mfx_sim.tfs_list:
    print('crl {} ({}um ROC) inserted: {}'.format(crl.name,crl.roc*1e6,crl.enabled))

crl tfs_2 (500.0um ROC) inserted: True
crl tfs_3 (300.0um ROC) inserted: False
crl tfs_4 (250.0um ROC) inserted: False
crl tfs_5 (200.0um ROC) inserted: False
crl tfs_6 (125.0um ROC) inserted: True
crl tfs_7 (62.5um ROC) inserted: True
crl tfs_8 (50.0um ROC) inserted: False
crl tfs_9 (50.0um ROC) inserted: False
crl tfs_10 (50.0um ROC) inserted: False


In [33]:
mfx_sim.tfs_2_x.wm()

-inf

In [29]:
mfx_sim.beam_params['ax']

0